In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision
# Örnek bir transform tanımla (istediğin şekilde düzenleyebilirsin)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Veri setini yükle
dataset = datasets.ImageFolder(root='./Kidney-Cancer', transform=transform)
BATCH_SIZE = 16
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

# Eğitim verisini %10 doğrulama için ayır
val_size = int(0.1 * len(train_set))
train_size -= val_size  # Geriye kalan kısım eğitim verisi olacak

train_set, val_set = torch.utils.data.random_split(train_set, [train_size, val_size])

# DataLoader'ları oluştur (batch_size'i ihtiyacınıza göre ayarlayabilirsin)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)


C:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT  
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model.classifier  = nn.Linear(1280,len(dataset.classes))

saved_weights = "./transfer-learning-1.pth"

model.load_state_dict(torch.load(saved_weights))

<All keys matched successfully>

In [8]:
lossF = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)


model.eval().to(device)
total_loss = 0
acc = 0

labelS = []
preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        results = model(inputs)
        loss = lossF(results, labels)
        total_loss += loss.item()
        _, predicted = torch.max(results, 1)
        acc += (predicted == labels).sum().item()
        
        
        labelS += labels.cpu().numpy().tolist()
        preds += predicted.cpu().numpy().tolist()
        
        
total_loss /= len(test_loader.dataset)
accuracy = acc / len(test_loader.dataset) * 100.0

print(f'Test Loss: {total_loss}, Test Accuracy: {accuracy}%')

Test Loss: 0.0005667432053305674, Test Accuracy: 99.8%


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
#Test verisinin Accuracy, 
#Precision, Recall ve F1 skoru, Karmaşıklık Matrisi, Classification Report
print("Accuracy score : {}".format(accuracy_score(labelS,preds)))
print("Precision score : {}".format(precision_score(labelS,preds,average="weighted")))
print("Recall score : {} ".format(recall_score(labelS, preds, average='weighted')))
print("\nConfusion matrix :\n {}".format(confusion_matrix(labelS,preds)))
print("\nClassification Report : \n{}".format(classification_report(labelS,preds,target_names=[str(i) for i in range(len(dataset.classes))])))

Accuracy score : 0.998
Precision score : 0.9980079443892751
Recall score : 0.998 

Confusion matrix :
 [[1003    0]
 [   4  993]]

Classification Report : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1003
           1       1.00      1.00      1.00       997

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

